In [1]:
!pip install transformers accelerate datasets torch torchvision peft pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall:

In [15]:
from datasets import load_dataset, DatasetDict
from transformers import (
AutoTokenizer,
AutoModelForCausalLM,
Trainer,
TrainingArguments
)
import torch
from peft import (
LoraConfig,
get_peft_model,
TaskType,
PeftConfig,
PeftModel
)
from huggingface_hub import notebook_login

In [3]:
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

In [4]:
ds = load_dataset("MCINext/synthetic-persian-qa-pair-classification")
ds

README.md: 0.00B [00:00, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


train.json:   0%|          | 0.00/949M [00:00<?, ?B/s]

test.json:   0%|          | 0.00/105M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/500106 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/55568 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sent1', 'sent2', 'labels'],
        num_rows: 500106
    })
    test: Dataset({
        features: ['sent1', 'sent2', 'labels'],
        num_rows: 55568
    })
})

In [5]:
ds = ds.remove_columns(["labels"])
ds

DatasetDict({
    train: Dataset({
        features: ['sent1', 'sent2'],
        num_rows: 500106
    })
    test: Dataset({
        features: ['sent1', 'sent2'],
        num_rows: 55568
    })
})

In [13]:
ds["train"][0]["sent2"]

'این قابلیت ارسال مسیر از نسخه رومیزی نقشه گوگل به دستگاه\u200cهای اندرویدی از مرورگرهای کروم، فایرفاکس و اینترنت اکسپلورر پشتیبانی می\u200cکند. کاربران می\u200cتوانند با استفاده از هر یک از این مرورگرها، مسیرهای جستجو شده را به دستگاه\u200cهای اندرویدی خود ارسال کنند و از این ویژگی جدید بهره\u200cمند شوند.'

In [14]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B-Instruct")

def preprocess(sample):
    sample = sample["sent1"] + "\n" + sample["sent2"]
    tokenized = tokenizer(
        sample,
        max_length = 128,
        truncation = True,
        padding = "max_length"
    )

    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [19]:
ds_small = DatasetDict({
    "train": ds["train"].select(range(1300))
})

In [20]:
ds_small

DatasetDict({
    train: Dataset({
        features: ['sent1', 'sent2'],
        num_rows: 1300
    })
})

In [21]:
data = ds_small.map(preprocess)
data

Map:   0%|          | 0/1300 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sent1', 'sent2', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1300
    })
})

In [22]:
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-3B-Instruct",
    device_map = "auto",
    offload_folder = "offload",
    torch_dtype = torch.float16
)

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [23]:
model.gradient_checkpointing_enable()

In [24]:
lora_config = LoraConfig(
    task_type = TaskType.CAUSAL_LM,
    target_modules = ["q_proj", "k_proj", "v_proj"]
)

In [25]:
model = get_peft_model(model, lora_config)

In [26]:
training_args = TrainingArguments(
    num_train_epochs = 27,
    learning_rate = 0.001,
    logging_steps = 100,
    #per_device_train_batch_size = 1,
    report_to = "tensorboard"
)

In [27]:
trainer = Trainer(
    model = model,
    train_dataset = data["train"],
    args = training_args
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [28]:
print("Start training...")
trainer.train()
print("Training finished...")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Start training...


Step,Training Loss
100,1.513500
200,1.407100
300,1.322000
400,1.212600
500,1.170200
600,1.029900
700,0.971200
800,0.895400
900,0.777700
1000,0.753600


Training finished...


In [29]:
trainer.save_model("/kaggle/working/")
tokenizer.save_pretrained("/kaggle/working/")

('/kaggle/working/tokenizer_config.json',
 '/kaggle/working/special_tokens_map.json',
 '/kaggle/working/chat_template.jinja',
 '/kaggle/working/vocab.json',
 '/kaggle/working/merges.txt',
 '/kaggle/working/added_tokens.json',
 '/kaggle/working/tokenizer.json')

In [30]:
path = "/kaggle/working/"

In [31]:
config = PeftConfig.from_pretrained(path)
base = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, trust_remote_code = True)
model = PeftModel.from_pretrained(base, path)
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code = True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [39]:
def generate_response(query):
    inputs = tokenizer(query, return_tensors = "pt").to(model.device)
    output = model.generate(
        input_ids = inputs["input_ids"],
        attention_mask = inputs["attention_mask"],
        max_new_tokens = 256,
        temperature = 0.6
    )
    return tokenizer.decode(output[0], skip_special_tokens = True)

In [36]:
print(generate_response("چرا رویکرد چابکی در مدیریت زنجیره تامین اهمیت دارد؟"))

چرا رویکرد چابکی در مدیریت زنجیره تامین اهمیت دارد؟
رویکرد چابکی در مدیریت زنجیره تامین به سازمان‌ها این امکان را می‌دهد که به سرعت به تغییرات تقاضا و شرایط بازار پاسخ دهند. این رویکرد به ویژه در بازارهای متلاطم و ناپایدار اهمیت دارد زیرا از طریق این طرح، سازمان‌ها می‌توانند تقاضای خود را ادامه دهند اما با استفاده از بنیادهای زنجیره تامین قبلی، که ممکن است در بازارهای همسایه انتخاب شده باشد.


In [38]:
print(generate_response("چگونه آب پرتقال می‌تواند به درمان اسهال در کودکان کمک کند؟"))

چگونه آب پرتقال می‌تواند به درمان اسهال در کودکان کمک کند؟
دلایل احتمالی اسهال در نوزادان می‌تواند شامل آلرژی یا حساسیت غذایی، مصرف لبنیات مانند شیر و مسمومیت غذایی باشد. شناسایی این دلایل می‌تواند به والدین کمک کند تا اقدامات لازم برای درمان اسهال از جمله مداوم سرشار سیاه‌چاله از الکل و مصرف مواد خطرناک را از دست بدهد.


In [40]:
print(generate_response("چرا خورشید اینقدر داغ است؟"))

چرا خورشید اینقدر داغ است؟
خورشید به دلیل فشار بسیار زیاد در هسته‌ی خود داغ است که باعث ایجاد همجوشی هسته‌ای مداوم می‌شود. این فرآیند گرما، نور و انرژی تولید می‌کند که به سمت سطح خورشید و سپس به فضای بیرونی تابش می‌شود. فرآیندهای شیمیایی و پایداری خورشید به این طور است که به ماه کلیدن یا رعایتی فراهم می‌شود.


In [41]:
print(generate_response("ضدگرانش چیست و چرا دانشمندان سرن به دنبال آن هستند؟"))

ضدگرانش چیست و چرا دانشمندان سرن به دنبال آن هستند؟
مادز تودال فراندسن، یکی از محققان، معتقد است که اگر سرن دارای شتاب‌دهنده قدرتمندتری باشد، باید قادر باشد تکنی‌کوارک‌ها را به طور مستقیم مشاهده کند. این مشاهده می‌تواند به دانشمندان این اطلاعیه جدیدی درباره شکل‌گیری مادز داده کند.


In [42]:
notebook_login()

In [43]:
model.push_to_hub("alikhademi98/finetuned_Qwen2.5_on_synthetic-persian-qa-pair-classification")
tokenizer.push_to_hub("alikhademi98/finetuned_Qwen2.5_on_synthetic-persian-qa-pair-classification")

Uploading...:   0%|          | 0.00/10.1M [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

Uploading...:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/alikhademi98/finetuned_Qwen2.5_on_synthetic-persian-qa-pair-classification/commit/94c385390b71b39955461e40bf7babbec189d7a6', commit_message='Upload tokenizer', commit_description='', oid='94c385390b71b39955461e40bf7babbec189d7a6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/alikhademi98/finetuned_Qwen2.5_on_synthetic-persian-qa-pair-classification', endpoint='https://huggingface.co', repo_type='model', repo_id='alikhademi98/finetuned_Qwen2.5_on_synthetic-persian-qa-pair-classification'), pr_revision=None, pr_num=None)